<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Chisel Demo
**Next: [Introduction to Scala](1_intro_to_scala)**

## Motivation
Perhaps you're an interested student who heard the name "Chisel" tossed about, or maybe you're a seasoned hardware design veteran who has been tasked by your manager to explore Chisel as a new HDL alternative. Either way if you are new to Chisel, you want to figure out as fast as possible what all the fuss is about. Look no futher - let's see what Chisel has to offer!

## Setup
The following cell downloads the dependencies needed for Chisel. You will see it in all future notebooks. **Run this cell now**.

In [7]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/adami/code/chisel3-bootcamp/chisel-bootcamp-demo/source/load-ivy.sc"

These statements are needed to import Chisel. **Run this cell now** before running any future code blocks.

In [8]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Bringing the Power of Parameterizability and Abstraction to Digital Design

Chisel is powered by [FIRRTL (Flexible Intermediate Representation for RTL)](https://github.com/freechipsproject/firrtl), a hardware compiler framework that performs optimizations of Chisel-generated circuits and supports custom user-defined circuit transformations.

## What does Chisel code look like?

Consider an FIR filter that implements a convolution operation, as depicted in this block diagram:

<img src="https://raw.githubusercontent.com/freechipsproject/chisel3/master/doc/images/fir_filter.svg?sanitize=true" width="512" />

While Chisel provides similar base primitives as synthesizable Verilog, and *could* be used as such:

In [11]:
// 3-point moving average implemented in the style of a FIR filter
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in)
  val z2 = RegNext(z1)

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U)
}

defined class MovingAverage3

the power of Chisel comes from the ability to create generators, such as n FIR filter that is defined by the list of coefficients:

In [12]:
// Generalized FIR filter parameterized by the convolution coefficients
class FirFilter(bitWidth: Int, coeffs: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i))

  // Sum up the products
  io.out := products.reduce(_ + _)
}

defined class FirFilter

and use and re-use them across designs:

In [21]:
// same 3-point moving average filter as before
visualize(() => new FirFilter(8, Seq(1.U, 1.U, 1.U)))
println(getVerilog(new FirFilter(8, Seq(1.U, 1.U, 1.U))))

[info] [0.000] Elaborating design...
[info] [0.029] Done elaborating.
Total FIRRTL Compile Time: 12.4 ms
creating dot file build/FirFilter.dot
print file closed 165 lines printed


[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 30.9 ms
module FirFilter(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  reg [7:0] zs_0; // @[cmd11.sc 7:15]
  reg [31:0] _RAND_0;
  reg [7:0] zs_1; // @[cmd11.sc 7:15]
  reg [31:0] _RAND_1;
  reg [7:0] zs_2; // @[cmd11.sc 7:15]
  reg [31:0] _RAND_2;
  wire [8:0] products_0; // @[cmd11.sc 14:61]
  wire [8:0] products_1; // @[cmd11.sc 14:61]
  wire [8:0] products_2; // @[cmd11.sc 14:61]
  wire [8:0] _T_4; // @[cmd11.sc 17:31]
  wire [8:0] _T_6; // @[cmd11.sc 17:31]
  assign products_0 = zs_0 * 8'h1; // @[cmd11.sc 14:61]
  assign products_1 = zs_1 * 8'h1; // @[cmd11.sc 14:61]
  assign products_2 = zs_2 * 8'h1; // @[cmd11.sc 14:61]
  assign _T_4 = products_0 + products_1; // @[cmd11.sc 17:31]
  assign _T_6 = _T_4 + products_2; // @[cmd11.sc 17:31]
  assign io_out = _T_6[7:0]; // @[cmd11.sc 17:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`e

In [20]:
// 1-cycle delay as a FIR filter
visualize(() => new FirFilter(8, Seq(0.U, 1.U)))

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 11.0 ms
creating dot file build/FirFilter.dot
print file closed 111 lines printed


In [22]:
// 5-point FIR filter with a triangle impulse response
visualize(() => new FirFilter(8, Seq(1.U, 2.U, 3.U, 2.U, 1.U)))

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 13.9 ms
creating dot file build/FirFilter.dot
print file closed 297 lines printed


# Transforming Designs

# Visualizing Designs

---
# You're done!

[Return to the top.](#top)